In [27]:
import pathlib
import sys
import shutil
import tempfile
import os
import subprocess
import csv
import biom
import numpy as np
import pandas as pd

from qiime2 import Artifact, Metadata, Visualization
from qiime2.plugins import cutadapt as cut
from qiime2.plugins.demux.visualizers import summarize
from qiime2.plugins.dada2.methods import denoise_single
from qiime2.plugins.feature_table.methods import filter_seqs
from qiime2.plugins.feature_table.methods import filter_features
from qiime2.plugins.feature_table.visualizers import tabulate_seqs
from qiime2.plugins.feature_table.visualizers import summarize as summarize_table

from remultiplexing import remultiplex
from index_jump import calculate_IJR
from index_jump import recalculate_IJR
from per_sample_filtering import per_sample_filter
from length_filtering import length_filter
from qiime2.plugins.feature_classifier.pipelines import classify_hybrid_vsearch_sklearn
from qiime2.plugins.feature_classifier.methods import classify_consensus_vsearch, classify_sklearn
from qiime2.plugins.taxa.visualizers import barplot
from qiime2.plugins.phylogeny.pipelines import align_to_tree_mafft_fasttree
from qiime2.plugins.diversity.pipelines import core_metrics_phylogenetic

def extract_tsv(file, dest):
    with tempfile.TemporaryDirectory() as temp:
        file.export_data(temp)
        temp_pathlib = pathlib.Path(temp)
        for file in temp_pathlib.iterdir():
            if file.suffix == '.tsv':
                shutil.copy(file, dest)


In [ ]:
#Remultiplex(name of bam file)
remultiplex('subsample.bam')

In [ ]:
#Gzip the all_seqs.fastq file produced during remultiplexing 
os.system('gzip all_seqs.fastq')

In [2]:
#Read in sample map
sample_map = 'Sample_Map_Full.txt'


metadata = Metadata.load(sample_map)
metadata_df = pd.read_csv(sample_map, sep='\t')

In [ ]:
#Importing a gzipped fastq as a fully multiplexed file into qiime2 
mux = Artifact.import_data("MultiplexedSingleEndBarcodeInSequence", "all_seqs.fastq.gz")

#Saving as a qza
mux.save('mux.qza')


In [ ]:
#Demultiplexing the reads according to their barcodes 
demux, untrimmed = cut.methods.demux_single(mux, 
                                            metadata.get_column('BarcodeSequence'), 
                                            error_rate = 0)
demux.save('demux.qza')
d = summarize(demux)
d.visualization

In [ ]:
#Trimming ANML primers off the demultiplexed reads
ANML_primers = ["^GGTCAACAAATCATAAAGATATTGG...GGATTTGGAAATTGATTAGTWCCATC"]

trimmed_demux = cut.methods.trim_single(demux, 
                                        cores=16, 
                                        adapter = ANML_primers, 
                                        indels = True,
                                        minimum_length = 170, 
                                        discard_untrimmed = True)
trimmed_demux = trimmed_demux.trimmed_sequences
trimmed_demux.save('trimmed_demux.qza')



In [5]:
# Loading Saved Files
trimmed_demux = Artifact.load('trimmed_demux.qza')

In [6]:
#Calculate max Index Jump Rate using calculate_IJR

max_IJR = calculate_IJR(trimmed_demux, metadata, metadata_df)
max_IJR = round(max_IJR)

Maximum number of false reads expected in a single sample: 1.519759980930058


mkdir: cannot create directory ‘tsvs’: File exists


In [ ]:
#DADA2
table, rep_seqs, stats = denoise_single(trimmed_demux,
                                        trunc_len = 0, 
                                        n_threads = 0)
rep_seqs.save('rep_seqs.qza')
table.save('table.qza')
stats.save('stats.qza')

In [7]:
#Re-loading artifacts
rep_seqs = Artifact.load('rep_seqs.qza')
table = Artifact.load('table.qza')

In [8]:
#Visualize inital table after dada2
tabulated = tabulate_seqs(rep_seqs)
tabulated.visualization

<visualization: Visualization uuid: f021cf4b-2307-4021-98b7-c6574b3affc5>

In [9]:
#Tabulate inital seqs after dada2
summary = summarize_table(table)
summary.visualization

<visualization: Visualization uuid: 6003a9a3-9496-425d-a2c3-5f5e1de35603>

In [10]:
#RECALCULATE_IJR using recalculate_IJRj
re_max_IJR = recalculate_IJR(rep_seqs, table, metadata_df)
re_max_IJR = round(re_max_IJR)

Maximum number of false reads expected in a single sample: 0.0


mkdir: cannot create directory ‘csvs’: File exists


In [11]:
#Filtering by Length
rep_seqs_filt_v1, table_filt_v1 = length_filter(rep_seqs, table, 120)

mkdir: cannot create directory ‘fastas’: File exists


In [12]:
#Tabulate seqs to ensure filtering
tabulated = tabulate_seqs(rep_seqs_filt_v1.filtered_data)
tabulated.visualization

<visualization: Visualization uuid: 705d9e68-60c0-4baf-b12e-1c19e61a583f>

In [14]:
#Filtering by Frequency 
table_filt_v2 = per_sample_filter(max_IJR, table_filt_v1)

Could not import from biom table, trying from qza
File loaded! Filtering at 2 occurences per feature


In [15]:
#Summarize table to ensure filtering 
summary = summarize_table(table_filt_v2)
summary.visualization

<visualization: Visualization uuid: d47d4f89-2aee-411f-be39-a7d670b01433>

In [17]:
#Import taxonomy files 

NoAm_seqs = Artifact.load('../COI_classifiers/NoAm_V3_91721/NoAm_seqs_V3.qza')
NoAm_tax = Artifact.load('../COI_classifiers/NoAm_V3_91721/NoAm_tax_V3.qza')
NoAm_class = Artifact.load('../COI_classifiers/NoAm_V3_91721/NoAm_classifier_V3.qza')

In [21]:
#Taxonomy Assignment
vsearch_taxonomy =      classify_consensus_vsearch(query = rep_seqs_filt_v1.filtered_data,
                                           reference_reads = NoAm_seqs,
                                           reference_taxonomy = NoAm_tax,
                                           maxaccepts = 1,
                                           perc_identity = .99,
                                           query_cov = .99,
                                           strand = 'both',
                                           threads = 35)
#Save taxonomy
vsearch_taxonomy = vsearch_taxonomy.classification
vsearch_taxonomy.save('vsearch_taxonomy.qza')

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /tmp/qiime2-archive-s4kf97_z/ef2b1e84-f805-4440-8f10-58c0f505406e/data/dna-sequences.fasta --id 0.99 --query_cov 0.99 --strand both --maxaccepts 1 --maxrejects 0 --db /tmp/qiime2-archive-xrq7e9zd/715f2018-2aba-4187-a330-22a50ac55c26/data/dna-sequences.fasta --threads 35 --output_no_hits --blast6out /tmp/tmp4sy6c_6l



vsearch v2.7.0_linux_x86_64, 251.8GB RAM, 36 cores
https://github.com/torognes/vsearch

Reading file /tmp/qiime2-archive-xrq7e9zd/715f2018-2aba-4187-a330-22a50ac55c26/data/dna-sequences.fasta 100%
1148947858 nt in 1913186 seqs, min 53, max 1537, avg 601
Masking 100%
Counting k-mers 100%
Creating k-mer index 100%
Searching 100%
Matching query sequences: 410 of 625 (65.60%)


'vsearch_taxonomy.qza'

In [36]:
#Grab the list of Feature ID's which were not assigned with Vsearch 
os.system('mkdir tsvs')
extract_tsv(vsearch_taxonomy, 'tsvs')
vsearch_df = pd.read_csv('tsvs/taxonomy.tsv', sep = '\t')

mkdir: cannot create directory ‘tsvs’: File exists


In [38]:
#Selecting only the features which were assigned
#Then sending them to a CSV which will make up our exclusion metadata

features_to_exclude = vsearch_df[vsearch_df['Taxon'] != 'Unassigned']

features_to_exclude['Feature ID'].to_csv('Features-to-exclude.csv', index=False)

exclude = Metadata.load("Features-to-exclude.csv")


In [46]:
#Filter merged_seqs based on seqs_to_exclude
unassigned_seqs  = filter_seqs(rep_seqs_filt_v1.filtered_data, 
                               metadata = exclude,
                               exclude_ids = True)

unassigned_table  = filter_features(table_filt_v2, 
                               metadata = exclude,
                               exclude_ids = True)

In [49]:
#Naive Bayes Taxonomy Assignment 

sklearn_taxonomy = classify_sklearn(reads = unassigned_seqs.filtered_data,
                                    classifier = NoAm_class,
                                    n_jobs = -1,
                                    read_orientation = 'auto')



In [51]:
#Then merge taxonomies
os.system('mkdir tsvs_sklearn')
extract_tsv(sklearn_taxonomy.classification, 'tsvs_sklearn')


sklearn_df = pd.read_csv('tsvs_sklearn/taxonomy.tsv', sep = '\t')

vsearch_df = features_to_exclude

frames = [vsearch_df, sklearn_df]

merged_taxonomies = pd.concat(frames,ignore_index=True)

merged_taxonomies = merged_taxonomies.set_index('Feature ID')
 
merged_taxonomy = Artifact.import_data("FeatureData[Taxonomy]", merged_taxonomies)

merged_taxonomy.save('merged_taxonomy.qza')

mkdir: cannot create directory ‘tsvs_sklearn’: File exists


'merged_taxonomy.qza'

In [53]:
#Barplot Creation
taxonomy = Artifact.load('merged_taxonomy.qza')
barplot = barplot(table_filt_v2, merged_taxonomy, metadata)
barplot = barplot.visualization
barplot.save('merged_barplot.qzv')
barplot = Visualization.load('merged_barplot.qzv')
barplot

<visualization: Visualization uuid: 66a1855b-09f6-4634-90ad-d2f3d803a2c8>

In [ ]:
# #Filter out low Zono, P, and C level IDs from table and seqs
# filter_list = ["k__Animalia;p__Chordata;c__Aves;o__Passeriformes;f__Passerellidae;g__Zonotrichia;s__albicollis",
#                "k__Animalia;p__Arthropoda;c__;o__;f__;g__;s__",
#                "k__Animalia;p__Arthropoda;__;__;__;__;__",
#                "k__Animalia;p__Arthropoda",
#                "k__Animalia;p__Arthropoda;c__Insecta;o__;f__;g__;s__",
#                "k__Animalia;p__Arthropoda;c__Insecta;__;__;__;__",
#                "k__Animalia;p__Arthropoda;c__Insecta",
#                "k__Animalia;p__Arthropoda;c__Arachnida;o__;f__;g__;s__",
#                "k__Animalia;p__Arthropoda;c__Arachnida;__;__;__;__",
#                "k__Animalia;p__Arthropoda;c__Arachnida",
#                "k__Animalia;p__Arthropoda;c__Collembola"]

# for i in filter_list:
